In [ ]:
from lxt.models.llama import LlamaForCausalLM, attnlrp
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from lxt.models.llama import LlamaForCausalLM, attnlrp

# load model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_id,device_map="cpu")


# apply LXT to the model
lxt_model = attnlrp.register(model)

# (optionally enable gradient checkpointing)
lxt_model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

In [ ]:
embeddings=None
def hook_fn(module, input, output):
    global embeddings
    output.requires_grad_(True)
    output.retain_grad() 
    embeddings=output

In [ ]:
hook = lxt_model.model.embed_tokens.register_forward_hook(hook_fn)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
prompt = """\
Context: Mount Everest attracts many climbers, including highly experienced mountaineers. There are two main climbing routes, one approaching the summit from the southeast in Nepal (known as the standard route) and the other from the north in Tibet. While not posing substantial technical climbing challenges on the standard route, Everest presents dangers such as altitude sickness, weather, and wind, as well as hazards from avalanches and the Khumbu Icefall. As of November 2022, 310 people have died on Everest. Over 200 bodies remain on the mountain and have not been removed due to the dangerous conditions. The first recorded efforts to reach Everest's summit were made by British mountaineers. As Nepal did not allow foreigners to enter the country at the time, the British made several attempts on the north ridge route from the Tibetan side. After the first reconnaissance expedition by the British in 1921 reached 7,000 m (22,970 ft) on the North Col, the 1922 expedition pushed the north ridge route up to 8,320 m (27,300 ft), marking the first time a human had climbed above 8,000 m (26,247 ft). The 1924 expedition resulted in one of the greatest mysteries on Everest to this day: George Mallory and Andrew Irvine made a final summit attempt on 8 June but never returned, sparking debate as to whether they were the first to reach the top. Tenzing Norgay and Edmund Hillary made the first documented ascent of Everest in 1953, using the southeast ridge route. Norgay had reached 8,595 m (28,199 ft) the previous year as a member of the 1952 Swiss expedition. The Chinese mountaineering team of Wang Fuzhou, Gonpo, and Qu Yinhua made the first reported ascent of the peak from the north ridge on 25 May 1960. \
Question: How high did they climb in 1922? According to the text, the 1922 expedition reached 8,"""

input_ids = tokenizer(prompt, return_tensors="pt")

In [ ]:
embeddings

In [ ]:
output_logits = lxt_model(**input_ids)

In [ ]:
embeddings

In [ ]:


# select token to explain
select_class_logit = output_logits.logits[0, -1, :].max()

# run backward
select_class_logit.backward(select_class_logit)

# obtain relevances by summing over embedding dimension i.e. keeping sequence dimension
relevance = embeddings.grad.float().sum(-1)

In [ ]:
from lxt.utils import pdf_heatmap, clean_tokens

# convert token ids to strings
tokens = tokenizer.convert_ids_to_tokens(input_ids.input_ids[0])

# removes the '_' character of tokens
tokens = clean_tokens( [token.replace('Ġ', '▁') for token in tokens])

# normalize relevance between [-1, 1] for plotting
relevance = relevance[0]
relevance = relevance / relevance.abs().max()

# generate PDF file
pdf_heatmap(tokens, relevance, path='heatmap.pdf', backend='xelatex')

In [ ]:
relevance